In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import functools
import ast
import keras
import pickle
from keras.models import Sequential
from keras.layers import Input,Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D,MaxPool1D,GlobalAveragePooling1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy 
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Input, Concatenate, BatchNormalization, MaxPooling1D, Embedding

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
graph = tf.get_default_graph()

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
session = keras.backend.get_session()
session.run(init)

Default GPU Device: /device:GPU:0


In [3]:
generic = lambda x: ast.literal_eval(x)
conv = {'Tags': generic}
df = pd.read_csv('data/data_final.csv', converters=conv)

In [4]:
sample = df.sample(frac=0.2, random_state=2020)
x1 = sample['Body']
y1 = sample['Tags']
max_length = 200

In [5]:
char_table = sorted(list(set(''.join(df['Body'].values))))
vocab_size = len(char_table)

In [6]:
x2 = x1.reset_index(drop=True)

In [7]:
xs = []
for enum, code in enumerate(x2):
    try:
        code_char =[]
        for char in code[:max_length]:
            if char in char_table:
                code_char.append(char_table.index(char))
        code_char_index = code_char + ([0] * (max_length - len(code)))
        xs.append(np.array(code_char_index))
    except:
        print(enum, code)
x = np.array(xs)

In [8]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit_transform(y1)

# saving
with open('binarizer.pickle', 'wb') as handle:
    pickle.dump(multilabel_binarizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('binarizer.pickle', 'rb') as handle:
    multilabel_binarizer = pickle.load(handle)

y = multilabel_binarizer.transform(y1)

In [9]:
print(y.shape)
print(x.shape)
print(vocab_size)
print(max_length)

(148964, 342)
(148964, 200)
4435
200


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=9000)

In [11]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=9000)

In [12]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(119171, 200)
(14896, 200)
(14897, 200)


In [18]:
#Replicated model
inputs = Input(shape=(max_length, ))
embedding = Embedding(vocab_size, 16, input_length=max_length, trainable=False)(inputs)
convs = []
for n, fsz in [(128, 2), (192, 3), (256, 4), (512, 5)]:
    model = Conv1D(n, fsz, activation='relu', padding='same')(embedding)
    model = BatchNormalization()(model)
    model = Conv1D(n, fsz, activation='relu', padding='same')(model)
    model = MaxPooling1D()(model)
    convs.append(model)

model = Concatenate(axis=-1)(convs)
model = Flatten()(model)
## Fully connected layers
model = BatchNormalization()(model)
model = Dense(7, activation='relu')(model)
model = BatchNormalization()(model)
model = Dense(7, activation='relu')(model)
model = BatchNormalization()(model)

model = Dense(y_train.shape[1], activation='sigmoid')(model)
model = Model(inputs, model)


1.13.1
Instructions for updating:
Colocations handled automatically by placer.


In [19]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 16)      70960       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 200, 128)     4224        embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 200, 192)     9408        embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_4 (

In [20]:
top1_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=1)
top5_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k = 5)
top1_acc.__name__ = 'top1_acc'
top5_acc.__name__ = 'top5_acc'

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc 

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy', 'top_k_categorical_accuracy', top1_acc, top5_acc, auc]
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [21]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
global session
global graph
with graph.as_default():
    set_session(session)
    history = model.fit(x_train, y_train,
                          epochs=10,
                          batch_size=16,
                          verbose=1,
                          validation_data=(x_val, y_val))

Train on 119171 samples, validate on 14896 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
119171/119171 [==============================] - 351s 3ms/sample - loss: 0.0532 - acc: 0.9901 - top_k_categorical_accuracy: 0.3227 - top1_acc: 0.1102 - top5_acc: 0.3227 - auc: 0.6655 - val_loss: 0.0287 - val_acc: 0.9939 - val_top_k_categorical_accuracy: 0.3841 - val_top1_acc: 0.1427 - val_top5_acc: 0.3841 - val_auc: 0.7492
Epoch 2/10
119171/119171 [==============================] - 348s 3ms/sample - loss: 0.0270 - acc: 0.9943 - top_k_categorical_accuracy: 0.4375 - top1_acc: 0.2021 - top5_acc: 0.4375 - auc: 0.7749 - val_loss: 0.0264 - val_acc: 0.9942 - val_top_k_categorical_accuracy: 0.4630 - val_top1_acc: 0.2328 - val_top5_acc: 0.4630 - val_auc: 0.7932
Epoch 3/10
119171/119171 [==============================] - 355s 3ms/sample - loss: 0.0252 - acc: 0.9944 - top_k_categorical_accuracy: 0.4953 - top1_acc: 0.2581 - top5_acc: 0.4953 - auc: 0.8057 - val_loss: 0.0277 - val_acc: 0.993

In [22]:
model.save("model/replicated_model_char_embedding.h5")

## Load model here

In [13]:
top1_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=1)
top5_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k = 5)
top1_acc.__name__ = 'top1_acc'
top5_acc.__name__ = 'top5_acc'

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc 

In [17]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
global session
global graph
with graph.as_default():
    set_session(session)
    model = tf.keras.models.load_model("model/replicated_model_char_embedding.h5", 
                                custom_objects={'top1_acc':top1_acc, 'top5_acc':top5_acc, 'auc':auc})

In [18]:
score = model.evaluate(x_test,y_test)

14897/14897 [==============================] - 15s 986us/sample - loss: 0.0242 - acc: 0.9945 - top_k_categorical_accuracy: 0.5500 - top1_acc: 0.3029 - top5_acc: 0.5500 - auc: 0.8724


In [19]:
for i in range (len(model.metrics_names)):
    print("Test {}: {}".format(model.metrics_names[i],score[i]) )

Test loss: 0.02418637656802226
Test acc: 0.9944603443145752
Test top_k_categorical_accuracy: 0.5499716401100159
Test top1_acc: 0.30287885665893555
Test top5_acc: 0.5499716401100159
Test auc: 0.8723740577697754
